In [ ]:

!pip install numpy opencv-python


In [ ]:
import numpy as np
import cv2

class SeamCarver:
    def __init__(self, image):
        self.original = image.copy()
        self.current = image.copy()
        self.height, self.width = image.shape[:2]

        # Initialize coordinate mapping and energy buffers
        self.x_mappings = np.tile(np.arange(self.width), (self.height, 1))
        self.removed_seams = []
        self._init_buffers()  # Initialize all buffers based on current image

    def _init_buffers(self):
        """Reinitialize all buffers to match current image dimensions"""
        # Convert to grayscale
        if self.current.ndim == 3:
            self.gray = cv2.cvtColor(self.current, cv2.COLOR_BGR2GRAY).astype(np.float32)
        else:
            self.gray = self.current.astype(np.float32)

        # Create fresh energy buffers with correct dimensions
        self.dx = np.empty_like(self.gray)
        self.dy = np.empty_like(self.gray)
        self.energy = np.empty_like(self.gray)

    def _compute_energy(self):
        """Compute energy with safe edge handling"""
        h, w = self.height, self.width

        # Horizontal gradient (central differences)
        if w > 2:  # Only compute central differences if width allows
            self.dx[:, 1:-1] = self.gray[:, 2:] - self.gray[:, :-2]
        if w >= 2:  # Handle edges safely
            self.dx[:, 0] = self.gray[:, 1] - self.gray[:, 0]
            self.dx[:, -1] = self.gray[:, -1] - self.gray[:, -2]

        # Vertical gradient (central differences)
        if h > 2:
            self.dy[1:-1, :] = self.gray[2:, :] - self.gray[:-2, :]
        if h >= 2:
            self.dy[0, :] = self.gray[1, :] - self.gray[0, :]
            self.dy[-1, :] = self.gray[-1, :] - self.gray[-2, :]

        np.abs(self.dx, out=self.dx)
        np.abs(self.dy, out=self.dy)
        np.add(self.dx, self.dy, out=self.energy)
        return self.energy

    def _compute_cumulative_energy(self):
        cum_energy = self.energy.copy()
        for i in range(1, self.height):
            left = np.roll(cum_energy[i-1], 1)
            left[0] = np.inf
            right = np.roll(cum_energy[i-1], -1)
            right[-1] = np.inf
            cum_energy[i] += np.minimum(np.minimum(left, cum_energy[i-1]), right)
        return cum_energy

    def _find_vertical_seam(self, cum_energy):
        seam = np.zeros(self.height, dtype=int)
        seam[-1] = np.argmin(cum_energy[-1])
        for i in range(self.height-2, -1, -1):
            j_prev = seam[i+1]
            j_min = max(j_prev-1, 0)
            j_max = min(j_prev+1, self.width-1)
            seam[i] = j_min + np.argmin(cum_energy[i, j_min:j_max+1])
        return seam

    def _remove_vertical_seam(self, seam):
        # Track original coordinates
        original_seam = [(i, int(self.x_mappings[i, seam[i]])) for i in range(self.height)]
        self.removed_seams.append(original_seam)

        # Update arrays using boolean mask
        mask = np.ones((self.height, self.width), dtype=bool)
        mask[np.arange(self.height), seam] = False

        self.current = self.current[mask].reshape((self.height, self.width-1, 3))
        self.x_mappings = self.x_mappings[mask].reshape((self.height, self.width-1))
        self.width -= 1

        # Reinitialize all buffers with new dimensions
        self._init_buffers()

    def carve_vertical(self, target_width):
        target_width = max(target_width, 1)
        while self.width > target_width:
            self._compute_energy()
            cum_energy = self._compute_cumulative_energy()
            seam = self._find_vertical_seam(cum_energy)
            self._remove_vertical_seam(seam)

    def visualize_seams(self):
        viz = self.original.copy()
        for seam in self.removed_seams:
            for row, col in seam:
                if 0 <= col < self.original.shape[1]:
                    viz[row, col] = [0, 0, 255]  # BGR color for red
        return viz

# Usage
if __name__ == "__main__":
    img = cv2.imread("images.jpeg")
    carver = SeamCarver(img)
    carver.carve_vertical(img.shape[1] // 1.5)
    cv2.imwrite("resized.jpg", carver.current)
    cv2.imwrite("seams.jpg", carver.visualize_seams())